# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=100)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=100)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(len(cities))

78


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
#Creating lists to hold values to later populate dataframe
lats = []
lons = []
max_temps = []
humidities = []
clouds = []
wind_speeds = []
countries =  []
dates = []
skip_counter = 0

#Beginning of print process
print("Beginning Data Retrieval")
print("------------------------------")

#Loop through each city in cities and generate json link to pull data
for name in cities:
    weather_url = f"https://api.openweathermap.org/data/2.5/weather?q={name}&appid={weather_api_key}&units=metric"
    response = requests.get(weather_url).json()

    #Catch exception in case no data found for selected city
    try:
        #Store data into variables
        lat = response["coord"]["lat"]
        lon = response["coord"]["lon"]
        max_temp = response["main"]["temp_max"]
        humidity = response["main"]["humidity"]
        cloudiness = response["clouds"]["all"]
        wind_speed = response["wind"]["speed"]
        country = response["sys"]["country"]
        date = response["dt"]

        #Append data to corresponding lists
        lats.append(lat)
        lons.append(lon)
        max_temps.append(max_temp)
        humidities.append(humidity)
        clouds.append(cloudiness)
        wind_speeds.append(wind_speed)
        countries.append(country)
        dates.append(date)

        #Print log of city being processed
        print(f"Processing record {cities.index(name)} | {name.capitalize()}")
        print(f"Data visual: {len(lats)}\n")
    except KeyError:
        print(f"No data found for {name.capitalize()}. Skipping...")

        #Remove city from cities list for easier dataframe creation later
        #cities.remove(name)
        lats.append("")
        lons.append("")
        max_temps.append("")
        humidities.append("")
        clouds.append("")
        wind_speeds.append("")
        countries.append("")
        dates.append("")
        print(f"Data visual: {len(lats)}\n")
        skip_counter += 1

#End of print process
print("-----------------------------")
print("Data Retrieval Complete")
print("-----------------------------")

print(f"Number of Skips: {skip_counter}")

Beginning Data Retrieval
------------------------------
Processing record 0 | Vaini
Data visual: 1

No data found for Illoqqortoormiut. Skipping...
Data visual: 2

Processing record 2 | Deputatskiy
Data visual: 3

No data found for Tuggurt. Skipping...
Data visual: 4

Processing record 4 | Busselton
Data visual: 5

Processing record 5 | Powell
Data visual: 6

Processing record 6 | Port hardy
Data visual: 7

Processing record 7 | Fortuna
Data visual: 8

Processing record 8 | Nabire
Data visual: 9

Processing record 9 | Punta arenas
Data visual: 10

Processing record 10 | Hilo
Data visual: 11

Processing record 11 | Los llanos de aridane
Data visual: 12

Processing record 12 | Bowen
Data visual: 13

Processing record 13 | Fuling
Data visual: 14

Processing record 14 | Mar del plata
Data visual: 15

Processing record 15 | Airai
Data visual: 16

Processing record 16 | Pilar
Data visual: 17

No data found for Umm jarr. Skipping...
Data visual: 18

Processing record 18 | Ushuaia
Data visual:

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [5]:
#Create dataframe
raw_data = {"City": cities, "Lat": lats, "Lon": lons, "Max Temp": max_temps, "Humidity": humidities, "Cloudiness": clouds, "Wind Speed": wind_speeds, "Country": countries, "Date": dates}

city_data = pd.DataFrame(raw_data)

#Find indexes where Lat is blank for future removal
index_names = city_data[city_data["Lat"] == ""].index

city_data.drop(index_names, inplace = True)

#Export dataframe to a csv file
city_data.to_csv("../output_data/cities.csv", index = False, header = True)

#Print out cleaned dataframe
city_data

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,vaini,-21.2,-175.2,21,83,40,3.1,TO,1602686579
2,deputatskiy,69.3,139.9,-10.57,98,99,1.34,RU,1602686579
4,busselton,-33.65,115.33,13.89,67,0,5.77,AU,1602686354
5,powell,40.16,-83.08,12.78,76,1,1.5,US,1602686580
6,port hardy,50.7,-127.42,8,100,90,2.38,CA,1602686528
...,...,...,...,...,...,...,...,...,...
73,norman wells,65.28,-126.83,-7,85,90,3.6,CA,1602686598
74,pemba,-12.97,40.52,26.19,75,0,10.13,MZ,1602686598
75,atuona,-9.8,-139.03,25.1,75,34,9.62,PF,1602686598
76,defiance,41.33,-84.45,9,81,1,3.73,US,1602686536


In [8]:
over_100 = city_data.loc[city_data["Humidity"] < 50]

over_100

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
28,sayyan,15.17,44.32,18.59,25,0,3.75,YE,1602686586
36,hermanus,-34.42,19.23,25,49,0,3.13,ZA,1602686468
72,najran,17.49,44.13,31,11,0,4.1,SA,1602686004


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [9]:
#  Get the indices of cities that have humidity over 100%.
humid_index = city_data[city_data["Humidity"] < 50].index
humid_index

Int64Index([28, 36, 72], dtype='int64')

In [10]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".
clean_city_data = city_data.drop(humid_index, inplace = False)

clean_city_data

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,vaini,-21.2,-175.2,21,83,40,3.1,TO,1602686579
2,deputatskiy,69.3,139.9,-10.57,98,99,1.34,RU,1602686579
4,busselton,-33.65,115.33,13.89,67,0,5.77,AU,1602686354
5,powell,40.16,-83.08,12.78,76,1,1.5,US,1602686580
6,port hardy,50.7,-127.42,8,100,90,2.38,CA,1602686528
...,...,...,...,...,...,...,...,...,...
73,norman wells,65.28,-126.83,-7,85,90,3.6,CA,1602686598
74,pemba,-12.97,40.52,26.19,75,0,10.13,MZ,1602686598
75,atuona,-9.8,-139.03,25.1,75,34,9.62,PF,1602686598
76,defiance,41.33,-84.45,9,81,1,3.73,US,1602686536


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression